Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff, LinearAlgebra

### snippet 10.4

In [2]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);
df[:pulled_left] = convert(Array{Int64}, df[:pulled_left])
df[:prosoc_left] = convert(Array{Int64}, df[:prosoc_left])
first(df, 5)

struct m_10_02d_model{TY <: AbstractVector, TX <: AbstractMatrix}
    "Observations."
    y::TY
    "Covariates"
    X::TX
    "Number of observations"
    N::Int
end

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::m_10_02d_model)(θ)
    @unpack y, X, N = problem   # extract the data
    @unpack β = θ  # works on the named tuple too
    ll = 0.0
    ll += sum(logpdf.(Normal(0, 10), β)) # a & bp
    ll += sum([loglikelihood(Binomial(1, logistic(dot(X[i, :], β))), [y[i]]) for i in 1:N])
    ll
end

Instantiate the model with data and inits.

In [4]:
N = size(df, 1)
X = hcat(ones(Int64, N), df[:prosoc_left]);
y = df[:pulled_left]
p = m_10_02d_model(y, X, N);
θ = (β = [1.0, 2.0],)
p(θ)

-487.6540051035728

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::m_10_02d_model) =
    as( (β = as(Array, size(p.X, 2)), ) )
# Wrap the problem with a transformation, then use Flux for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Tune and sample.

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.0027 s/step ...done
MCMC, adapting ϵ (25 steps)
0.0013 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0031 s/step ...done
MCMC, adapting ϵ (100 steps)
0.00078 s/step ...done
MCMC, adapting ϵ (200 steps)
0.00076 s/step ...done
MCMC, adapting ϵ (400 steps)
0.00068 s/step ...done
MCMC, adapting ϵ (50 steps)
0.00074 s/step ...done
MCMC (1000 steps)
0.00062 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β,),Tuple{Array{Float64,1}}},1}:
 (β = [0.0166965, 0.632433],)
 (β = [0.10658, 0.332437],)  
 (β = [0.0885895, 0.521233],)
 (β = [-0.137458, 0.640464],)
 (β = [0.016397, 0.637151],) 

Extract the parameter posterior means: `β`,

In [8]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
 0.046246919552426546
 0.5596657389044554  

Effective sample sizes (of untransformed draws)

In [9]:
ess = mapslices(effective_sample_size, get_position_matrix(chain); dims = 1)
ess

1×2 Array{Float64,2}:
 850.766  991.479

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.9, min/25%/median/75%/max: 0.41 0.84 0.94 0.99 1.0
  termination: AdjacentTurn => 23% DoubledTurn => 77%
  depth: 1 => 24% 2 => 75% 3 => 1%


CmdStan result

In [11]:
m_10_2s_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
      Mean        SD       Naive SE       MCSE      ESS
 a 0.05103234 0.12579086 0.0019889282 0.0035186307 1000
bp 0.55711212 0.18074275 0.0028577937 0.0040160451 1000

Quantiles:
       2.5%        25.0%       50.0%      75.0%      97.5%
 a -0.19755400 -0.029431425 0.05024655 0.12978825 0.30087758
bp  0.20803447  0.433720250 0.55340400 0.67960975 0.91466915
";

Extract the parameter posterior means: `β`,

In [12]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
 0.046246919552426546
 0.5596657389044554  

End of `10/m10.02d.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*